In [16]:
import numpy as np
TaiwanLatRange=np.arange(21.91456,25.29548,0.00676184)
TaiwanLngRange=np.arange(120.04193,122.00059,0.009793304)

In [23]:
TWLocs=[]
for i in TaiwanLatRange:
    for j in TaiwanLngRange:
        TWLoc={}
        TWLoc['lat']=i
        TWLoc['lng']=j
        TWLocs.append(TWLoc)

In [25]:
TWLocs

[{'lat': 21.91456, 'lng': 120.04193},
 {'lat': 21.91456, 'lng': 120.05172330399999},
 {'lat': 21.91456, 'lng': 120.06151660799999},
 {'lat': 21.91456, 'lng': 120.07130991199999},
 {'lat': 21.91456, 'lng': 120.08110321599999},
 {'lat': 21.91456, 'lng': 120.09089651999999},
 {'lat': 21.91456, 'lng': 120.10068982399999},
 {'lat': 21.91456, 'lng': 120.11048312799998},
 {'lat': 21.91456, 'lng': 120.12027643199998},
 {'lat': 21.91456, 'lng': 120.13006973599998},
 {'lat': 21.91456, 'lng': 120.13986303999998},
 {'lat': 21.91456, 'lng': 120.14965634399998},
 {'lat': 21.91456, 'lng': 120.15944964799998},
 {'lat': 21.91456, 'lng': 120.16924295199998},
 {'lat': 21.91456, 'lng': 120.17903625599997},
 {'lat': 21.91456, 'lng': 120.18882955999997},
 {'lat': 21.91456, 'lng': 120.19862286399997},
 {'lat': 21.91456, 'lng': 120.20841616799997},
 {'lat': 21.91456, 'lng': 120.21820947199997},
 {'lat': 21.91456, 'lng': 120.22800277599997},
 {'lat': 21.91456, 'lng': 120.23779607999997},
 {'lat': 21.91456, 'ln

In [26]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib

def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='AutoAddInsurance'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

try:
    ######################################################IPEEN
    b=time.time()
    queryElements = {}
    client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find(queryElements))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0
                 and dien['averagecost'] < 8000]

    for dien in ipeendata:
        dien["id"] = dien.pop("_id")
    ######################################################IPEEN
    ######################################################104
    collection = client.rawData.HRdata104
    hr104data = list(collection.find(queryElements))
    hr104data = [dien for dien in hr104data if dien['LAT'] > 18
                 and dien['LAT'] < 27
                 and dien['SAL_MONTH_LOW'] > 18000
                 and dien['SAL_MONTH_LOW'] < 100000
                 and dien['SAL_MONTH_HIGH'] > 18000
                 and dien['SAL_MONTH_HIGH'] < 200000
                 and dien['LON'] < 125
                 and dien['LON'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0]

    for dien in hr104data:
        dien["lat"] = dien.pop("LAT")
        dien["lng"] = dien.pop("LON")
        del dien["_id"]
    ######################################################104
    ######################################################Nhuman
    collection = client.rawData.Nhuman
    Nhumandata = list(collection.find(queryElements))
    for dien in Nhumandata:
        dien["weight"] = int(dien.pop("Nhuman"))
        dien["add"] = dien.pop("_id")
    ######################################################Nhuman
    ######################################################CostPower
    collection = client.rawData.CostPower
    CostPowerdata = list(collection.find(queryElements))
    for dien in CostPowerdata:
        dien["weight"] = int(dien.pop('costPower'))
        dien["add"] = dien.pop("_id")
    ######################################################CostPower
    ######################################################Bus0320
    collection = client.rawData.busData
    busData = list(collection.find(queryElements))
    for dien in busData:
        dien["add"] = dien.pop("_id")
    ######################################################Bus0320
    ######################################################conStore0320
    collection = client.rawData.conStore
    conStoreData = list(collection.find(queryElements))
    for dien in conStoreData:
        dien.pop("_id")
    ######################################################conStore0320
    ######################################################Wow
    wowDiensData = TWLocs

    x = []
    queryDien = ""#input("請輸入品牌名稱:")
    radius = 2000#int(input("半徑範圍(公尺):"))
    if queryDien != "":
        wowDiensData = [i for i in wowDiensData if i['Called'] == queryDien]
    else:
        queryDien = "全品牌"
    print("有{}筆資料要分析".format(len(wowDiensData)))
    n = 0
    for wowDien in wowDiensData:
        n += 1
        if n % 15 == 0:
            print("已完成{}項分析".format(n))
        wowDien["areaRadius_Analyze"] = radius
        dienCostPower = round(numpy.mean([dien['weight'] for dien in CostPowerdata if haversine(lng1=dien["lng"],
                                                                                                lat1=dien["lat"],
                                                                                                lng2=wowDien["lng"],
                                                                                                lat2=wowDien[
                                                                                                    "lat"]) <= radius and 'weight' in dien]))

        if not math.isnan(dienCostPower):
            wowDien["costPower_Analyze"] = dienCostPower
        else:
            wowDien["costPower_Analyze"] =0
        wowDien["NcostData_Analyze"] = len([dien['weight'] for dien in CostPowerdata if haversine(lng1=dien["lng"],
                                                                                lat1=dien["lat"],
                                                                                lng2=wowDien["lng"],
                                                                                lat2=wowDien["lat"]) <= radius and 'weight' in dien])
        
        dienHuman = sum([dien['weight'] for dien in Nhumandata if haversine(lng1=dien["lng"],
                                                                            lat1=dien["lat"],
                                                                            lng2=wowDien["lng"],
                                                                            lat2=wowDien["lat"]) <= radius and 'weight' in dien])
        wowDien['Nhuman_Analyze'] = dienHuman

        avgSalary = round(numpy.mean(
            [dien['SAL_MONTH_LOW'] * 1 / 3 + dien['SAL_MONTH_HIGH'] * 2 / 3 for dien in hr104data if
             haversine(lng1=dien["lng"],
                       lat1=dien["lat"],
                       lng2=wowDien["lng"],
                       lat2=wowDien["lat"]) <= radius and 'SAL_MONTH_HIGH' in dien and 'SAL_MONTH_LOW' in dien]))
        if not math.isnan(avgSalary):
            wowDien['avgSalary_Analyze'] = avgSalary
        else:
            wowDien["avgSalary_Analyze"] =0

        wowDien['Njob_Analyze'] = len([dien['SAL_MONTH_LOW'] * 1 / 3 + dien['SAL_MONTH_HIGH'] * 2 / 3 for dien in hr104data if
                            haversine(lng1=dien["lng"],
                                      lat1=dien["lat"],
                                      lng2=wowDien["lng"],
                                      lat2=wowDien["lat"]) <= radius and 'SAL_MONTH_HIGH' in dien  and 'SAL_MONTH_LOW' in dien])

        avgCost = round(numpy.mean([dien['averagecost'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                                           lat1=dien["lat"],
                                                                                           lng2=wowDien["lng"],
                                                                                           lat2=wowDien[
                                                                                               "lat"]) <= radius and 'averagecost' in dien]))
        if not math.isnan(avgCost):
            wowDien['avgCost_Analyze'] = avgCost
        else:
            wowDien["avgCost_Analyze"] =0

        try:
            mostStyle = Counter([dien['bigstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                                     lat1=dien["lat"],
                                                                                     lng2=wowDien["lng"],
                                                                                     lat2=wowDien[
                                                                                         "lat"]) <= radius and 'bigstyle' in dien]).most_common(1)[0][0]
        except:
            mostStyle = ""
        wowDien['mostStyle_Analyze'] = mostStyle
        
        ###############################################################################################################0320
        wowDien["NbusStation_Analyze"] = len([dien['stationName'] for dien in busData if "lng" in dien and haversine(lng1=dien["lng"],
                                                                                lat1=dien["lat"],
                                                                                lng2=wowDien["lng"],
                                                                                lat2=wowDien["lat"]) <= 500])#0326 change radius
        
        wowDien["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=wowDien["lng"],
                                                                                lat2=wowDien["lat"]) <= radius])
#         conStoreData
        wowDien["Nstar_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '星巴' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])
        wowDien["Nmc_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '麥當' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])
        wowDien["Nken_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '肯德' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])
        wowDien["Nwa_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '瓦城泰' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])
        ################################################################################################################0320
        
        
        
        
        e = time.time()
    print(e-b)


#     ids=[data.pop("_id") for data in wowDiensData]
#     operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,wowDiensData)]
#     try:
#         collection.bulk_write(operations ,ordered=False)
#     except Exception as er:
#         print(er)

#     alldata=list(collection.find({}))

#     client.close()
    
    mailTo(title="analyzeWowDataSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b),whoSend='analyzeWowData')
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="analyzeWowDataFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg,whoSend='analyzeWowData')

有100000筆資料要分析


D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


已完成15項分析
已完成30項分析
已完成45項分析
已完成60項分析
已完成75項分析
已完成90項分析
已完成105項分析
已完成120項分析
已完成135項分析
已完成150項分析
已完成165項分析
已完成180項分析
已完成195項分析
已完成210項分析
已完成225項分析
已完成240項分析
已完成255項分析
已完成270項分析
已完成285項分析
已完成300項分析
已完成315項分析
已完成330項分析
已完成345項分析
已完成360項分析
已完成375項分析
已完成390項分析
已完成405項分析
已完成420項分析
已完成435項分析
已完成450項分析
已完成465項分析
已完成480項分析
已完成495項分析
已完成510項分析
已完成525項分析
已完成540項分析
已完成555項分析
已完成570項分析
已完成585項分析
已完成600項分析
已完成615項分析
已完成630項分析
已完成645項分析
已完成660項分析
已完成675項分析
已完成690項分析
已完成705項分析
已完成720項分析
已完成735項分析
已完成750項分析
已完成765項分析
已完成780項分析
已完成795項分析
已完成810項分析
已完成825項分析
已完成840項分析
已完成855項分析
已完成870項分析
已完成885項分析
已完成900項分析
已完成915項分析
已完成930項分析
已完成945項分析
已完成960項分析
已完成975項分析
已完成990項分析
已完成1005項分析
已完成1020項分析
已完成1035項分析
已完成1050項分析
已完成1065項分析
已完成1080項分析
已完成1095項分析
已完成1110項分析
已完成1125項分析
已完成1140項分析
已完成1155項分析
已完成1170項分析
已完成1185項分析
已完成1200項分析
已完成1215項分析
已完成1230項分析
已完成1245項分析
已完成1260項分析
已完成1275項分析
已完成1290項分析
已完成1305項分析
已完成1320項分析
已完成1335項分析
已完成1350項分析
已完成1365項分析
已完成1380項分析
已完成1395項分析
已完成1410項分析
已完成1425項分析
已完成1440項分析
已完成1455項分析
已完成14

KeyboardInterrupt: 

In [75]:
for wowDien in wowDiensData:
    if "areaRadius_Analyze" not in wowDien:
        n += 1
        if n % 15 == 0:
            print("已完成{}項分析".format(n))
        if n % 1000 == 0:
            for i in wowDiensData:
                i["_id"]="{}_{}".format(i['lat'],i['lng'])
            with open("taiwan{}.json".format(n),"w") as f:
                json.dump(wowDiensData,f)
            client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfo
            ids=[data.pop("_id") for data in wowDiensData]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,wowDiensData)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()
            
        wowDien["areaRadius_Analyze"] = radius
        dienCostPower = round(numpy.mean([dien['weight'] for dien in CostPowerdata if haversine(lng1=dien["lng"],
                                                                                                lat1=dien["lat"],
                                                                                                lng2=wowDien["lng"],
                                                                                                lat2=wowDien[
                                                                                                    "lat"]) <= radius and 'weight' in dien]))

        if not math.isnan(dienCostPower):
            wowDien["costPower_Analyze"] = dienCostPower
        else:
            wowDien["costPower_Analyze"] =0
        wowDien["NcostData_Analyze"] = len([dien['weight'] for dien in CostPowerdata if haversine(lng1=dien["lng"],
                                                                                lat1=dien["lat"],
                                                                                lng2=wowDien["lng"],
                                                                                lat2=wowDien["lat"]) <= radius and 'weight' in dien])

        dienHuman = sum([dien['weight'] for dien in Nhumandata if haversine(lng1=dien["lng"],
                                                                            lat1=dien["lat"],
                                                                            lng2=wowDien["lng"],
                                                                            lat2=wowDien["lat"]) <= radius and 'weight' in dien])
        wowDien['Nhuman_Analyze'] = dienHuman

        avgSalary = round(numpy.mean(
            [dien['SAL_MONTH_LOW'] * 1 / 3 + dien['SAL_MONTH_HIGH'] * 2 / 3 for dien in hr104data if
             haversine(lng1=dien["lng"],
                       lat1=dien["lat"],
                       lng2=wowDien["lng"],
                       lat2=wowDien["lat"]) <= radius and 'SAL_MONTH_HIGH' in dien and 'SAL_MONTH_LOW' in dien]))
        if not math.isnan(avgSalary):
            wowDien['avgSalary_Analyze'] = avgSalary
        else:
            wowDien["avgSalary_Analyze"] =0

        wowDien['Njob_Analyze'] = len([dien['SAL_MONTH_LOW'] * 1 / 3 + dien['SAL_MONTH_HIGH'] * 2 / 3 for dien in hr104data if
                            haversine(lng1=dien["lng"],
                                      lat1=dien["lat"],
                                      lng2=wowDien["lng"],
                                      lat2=wowDien["lat"]) <= radius and 'SAL_MONTH_HIGH' in dien  and 'SAL_MONTH_LOW' in dien])

        avgCost = round(numpy.mean([dien['averagecost'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                                           lat1=dien["lat"],
                                                                                           lng2=wowDien["lng"],
                                                                                           lat2=wowDien[
                                                                                               "lat"]) <= radius and 'averagecost' in dien]))
        if not math.isnan(avgCost):
            wowDien['avgCost_Analyze'] = avgCost
        else:
            wowDien["avgCost_Analyze"] =0

        try:
            mostStyle = Counter([dien['bigstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                                     lat1=dien["lat"],
                                                                                     lng2=wowDien["lng"],
                                                                                     lat2=wowDien[
                                                                                         "lat"]) <= radius and 'bigstyle' in dien]).most_common(1)[0][0]
        except:
            mostStyle = ""
        wowDien['mostStyle_Analyze'] = mostStyle

        ###############################################################################################################0320
        wowDien["NbusStation_Analyze"] = len([dien['stationName'] for dien in busData if "lng" in dien and haversine(lng1=dien["lng"],
                                                                                lat1=dien["lat"],
                                                                                lng2=wowDien["lng"],
                                                                                lat2=wowDien["lat"]) <= 500])#0326 change radius

        wowDien["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=wowDien["lng"],
                                                                                lat2=wowDien["lat"]) <= radius])
    #         conStoreData
        wowDien["Nstar_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '星巴' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])
        wowDien["Nmc_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '麥當' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])
        wowDien["Nken_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '肯德' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])
        wowDien["Nwa_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '瓦城泰' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=wowDien["lng"],
                                              lat2=wowDien["lat"]) <= radius])

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


已完成16005項分析
已完成16020項分析
已完成16035項分析
已完成16050項分析
已完成16065項分析
已完成16080項分析
已完成16095項分析
已完成16110項分析
已完成16125項分析
已完成16140項分析
已完成16155項分析
已完成16170項分析
已完成16185項分析
已完成16200項分析
已完成16215項分析
已完成16230項分析
已完成16245項分析
已完成16260項分析
已完成16275項分析
已完成16290項分析
已完成16305項分析
已完成16320項分析
已完成16335項分析
已完成16350項分析
已完成16365項分析
已完成16380項分析
已完成16395項分析
已完成16410項分析
已完成16425項分析
已完成16440項分析
已完成16455項分析
已完成16470項分析
已完成16485項分析
已完成16500項分析
已完成16515項分析
已完成16530項分析
已完成16545項分析
已完成16560項分析
已完成16575項分析
已完成16590項分析
已完成16605項分析
已完成16620項分析
已完成16635項分析
已完成16650項分析
已完成16665項分析
已完成16680項分析
已完成16695項分析
已完成16710項分析
已完成16725項分析
已完成16740項分析
已完成16755項分析
已完成16770項分析
已完成16785項分析
已完成16800項分析
已完成16815項分析
已完成16830項分析
已完成16845項分析
已完成16860項分析
已完成16875項分析
已完成16890項分析
已完成16905項分析
已完成16920項分析
已完成16935項分析
已完成16950項分析
已完成16965項分析
已完成16980項分析
已完成16995項分析
已完成17010項分析
已完成17025項分析
已完成17040項分析
已完成17055項分析
已完成17070項分析
已完成17085項分析
已完成17100項分析
已完成17115項分析
已完成17130項分析
已完成17145項分析
已完成17160項分析
已完成17175項分析
已完成17190項分析
已完成17205項分析
已完成17220項分析
已完成17235項分析
已完成1

已完成26250項分析
已完成26265項分析
已完成26280項分析
已完成26295項分析
已完成26310項分析
已完成26325項分析
已完成26340項分析
已完成26355項分析
已完成26370項分析
已完成26385項分析
已完成26400項分析
已完成26415項分析
已完成26430項分析
已完成26445項分析
已完成26460項分析
已完成26475項分析
已完成26490項分析
已完成26505項分析
已完成26520項分析
已完成26535項分析
已完成26550項分析
已完成26565項分析
已完成26580項分析
已完成26595項分析
已完成26610項分析
已完成26625項分析
已完成26640項分析
已完成26655項分析
已完成26670項分析
已完成26685項分析
已完成26700項分析
已完成26715項分析
已完成26730項分析
已完成26745項分析
已完成26760項分析
已完成26775項分析
已完成26790項分析
已完成26805項分析
已完成26820項分析
已完成26835項分析
已完成26850項分析
已完成26865項分析
已完成26880項分析
已完成26895項分析
已完成26910項分析
已完成26925項分析
已完成26940項分析
已完成26955項分析
已完成26970項分析
已完成26985項分析
已完成27000項分析
已完成27015項分析
已完成27030項分析
已完成27045項分析
已完成27060項分析
已完成27075項分析
已完成27090項分析
已完成27105項分析
已完成27120項分析
已完成27135項分析
已完成27150項分析
已完成27165項分析
已完成27180項分析
已完成27195項分析
已完成27210項分析
已完成27225項分析
已完成27240項分析
已完成27255項分析
已完成27270項分析
已完成27285項分析
已完成27300項分析
已完成27315項分析
已完成27330項分析
已完成27345項分析
已完成27360項分析
已完成27375項分析
已完成27390項分析
已完成27405項分析
已完成27420項分析
已完成27435項分析
已完成27450項分析
已完成27465項分析
已完成27480項分析
已完成2

已完成36495項分析
已完成36510項分析
已完成36525項分析
已完成36540項分析
已完成36555項分析
已完成36570項分析
已完成36585項分析
已完成36600項分析
已完成36615項分析
已完成36630項分析
已完成36645項分析
已完成36660項分析
已完成36675項分析
已完成36690項分析
已完成36705項分析
已完成36720項分析
已完成36735項分析
已完成36750項分析
已完成36765項分析
已完成36780項分析
已完成36795項分析
已完成36810項分析
已完成36825項分析
已完成36840項分析
已完成36855項分析
已完成36870項分析
已完成36885項分析
已完成36900項分析
已完成36915項分析
已完成36930項分析
已完成36945項分析
已完成36960項分析
已完成36975項分析
已完成36990項分析
已完成37005項分析
已完成37020項分析
已完成37035項分析
已完成37050項分析
已完成37065項分析
已完成37080項分析
已完成37095項分析
已完成37110項分析
已完成37125項分析
已完成37140項分析
已完成37155項分析
已完成37170項分析
已完成37185項分析
已完成37200項分析
已完成37215項分析
已完成37230項分析
已完成37245項分析
已完成37260項分析
已完成37275項分析
已完成37290項分析
已完成37305項分析
已完成37320項分析
已完成37335項分析
已完成37350項分析
已完成37365項分析
已完成37380項分析
已完成37395項分析
已完成37410項分析
已完成37425項分析
已完成37440項分析
已完成37455項分析
已完成37470項分析
已完成37485項分析
已完成37500項分析
已完成37515項分析
已完成37530項分析
已完成37545項分析
已完成37560項分析
已完成37575項分析
已完成37590項分析
已完成37605項分析
已完成37620項分析
已完成37635項分析
已完成37650項分析
已完成37665項分析
已完成37680項分析
已完成37695項分析
已完成37710項分析
已完成37725項分析
已完成3

已完成46740項分析
已完成46755項分析
已完成46770項分析
已完成46785項分析
已完成46800項分析
已完成46815項分析
已完成46830項分析
已完成46845項分析
已完成46860項分析
已完成46875項分析
已完成46890項分析
已完成46905項分析
已完成46920項分析
已完成46935項分析
已完成46950項分析
已完成46965項分析
已完成46980項分析
已完成46995項分析
已完成47010項分析
已完成47025項分析
已完成47040項分析
已完成47055項分析
已完成47070項分析
已完成47085項分析
已完成47100項分析
已完成47115項分析
已完成47130項分析
已完成47145項分析
已完成47160項分析
已完成47175項分析
已完成47190項分析
已完成47205項分析
已完成47220項分析
已完成47235項分析
已完成47250項分析
已完成47265項分析
已完成47280項分析
已完成47295項分析
已完成47310項分析
已完成47325項分析
已完成47340項分析
已完成47355項分析
已完成47370項分析
已完成47385項分析
已完成47400項分析
已完成47415項分析
已完成47430項分析
已完成47445項分析
已完成47460項分析
已完成47475項分析
已完成47490項分析
已完成47505項分析
已完成47520項分析
已完成47535項分析
已完成47550項分析
已完成47565項分析
已完成47580項分析
已完成47595項分析
已完成47610項分析
已完成47625項分析
已完成47640項分析
已完成47655項分析
已完成47670項分析
已完成47685項分析
已完成47700項分析
已完成47715項分析
已完成47730項分析
已完成47745項分析
已完成47760項分析
已完成47775項分析
已完成47790項分析
已完成47805項分析
已完成47820項分析
已完成47835項分析
已完成47850項分析
已完成47865項分析
已完成47880項分析
已完成47895項分析
已完成47910項分析
已完成47925項分析
已完成47940項分析
已完成47955項分析
已完成47970項分析
已完成4

已完成56985項分析
已完成57000項分析
已完成57015項分析
已完成57030項分析
已完成57045項分析
已完成57060項分析
已完成57075項分析
已完成57090項分析
已完成57105項分析
已完成57120項分析
已完成57135項分析
已完成57150項分析
已完成57165項分析
已完成57180項分析
已完成57195項分析
已完成57210項分析
已完成57225項分析
已完成57240項分析
已完成57255項分析
已完成57270項分析
已完成57285項分析
已完成57300項分析
已完成57315項分析
已完成57330項分析
已完成57345項分析
已完成57360項分析
已完成57375項分析
已完成57390項分析
已完成57405項分析
已完成57420項分析
已完成57435項分析
已完成57450項分析
已完成57465項分析
已完成57480項分析
已完成57495項分析
已完成57510項分析
已完成57525項分析
已完成57540項分析
已完成57555項分析
已完成57570項分析
已完成57585項分析
已完成57600項分析
已完成57615項分析
已完成57630項分析
已完成57645項分析
已完成57660項分析
已完成57675項分析
已完成57690項分析
已完成57705項分析
已完成57720項分析
已完成57735項分析
已完成57750項分析
已完成57765項分析
已完成57780項分析
已完成57795項分析
已完成57810項分析
已完成57825項分析
已完成57840項分析
已完成57855項分析
已完成57870項分析
已完成57885項分析
已完成57900項分析
已完成57915項分析
已完成57930項分析
已完成57945項分析
已完成57960項分析
已完成57975項分析
已完成57990項分析
已完成58005項分析
已完成58020項分析
已完成58035項分析
已完成58050項分析
已完成58065項分析
已完成58080項分析
已完成58095項分析
已完成58110項分析
已完成58125項分析
已完成58140項分析
已完成58155項分析
已完成58170項分析
已完成58185項分析
已完成58200項分析
已完成58215項分析
已完成5

已完成67230項分析
已完成67245項分析
已完成67260項分析
已完成67275項分析
已完成67290項分析
已完成67305項分析
已完成67320項分析
已完成67335項分析
已完成67350項分析
已完成67365項分析
已完成67380項分析
已完成67395項分析
已完成67410項分析
已完成67425項分析
已完成67440項分析
已完成67455項分析
已完成67470項分析
已完成67485項分析
已完成67500項分析
已完成67515項分析
已完成67530項分析
已完成67545項分析
已完成67560項分析
已完成67575項分析
已完成67590項分析
已完成67605項分析
已完成67620項分析
已完成67635項分析
已完成67650項分析
已完成67665項分析
已完成67680項分析
已完成67695項分析
已完成67710項分析
已完成67725項分析
已完成67740項分析
已完成67755項分析
已完成67770項分析
已完成67785項分析
已完成67800項分析
已完成67815項分析
已完成67830項分析
已完成67845項分析
已完成67860項分析
已完成67875項分析
已完成67890項分析
已完成67905項分析
已完成67920項分析
已完成67935項分析
已完成67950項分析
已完成67965項分析
已完成67980項分析
已完成67995項分析
已完成68010項分析
已完成68025項分析
已完成68040項分析
已完成68055項分析
已完成68070項分析
已完成68085項分析
已完成68100項分析
已完成68115項分析
已完成68130項分析
已完成68145項分析
已完成68160項分析
已完成68175項分析
已完成68190項分析
已完成68205項分析
已完成68220項分析
已完成68235項分析
已完成68250項分析
已完成68265項分析
已完成68280項分析
已完成68295項分析
已完成68310項分析
已完成68325項分析
已完成68340項分析
已完成68355項分析
已完成68370項分析
已完成68385項分析
已完成68400項分析
已完成68415項分析
已完成68430項分析
已完成68445項分析
已完成68460項分析
已完成6

已完成77475項分析
已完成77490項分析
已完成77505項分析
已完成77520項分析
已完成77535項分析
已完成77550項分析
已完成77565項分析
已完成77580項分析
已完成77595項分析
已完成77610項分析
已完成77625項分析
已完成77640項分析
已完成77655項分析
已完成77670項分析
已完成77685項分析
已完成77700項分析
已完成77715項分析
已完成77730項分析
已完成77745項分析
已完成77760項分析
已完成77775項分析
已完成77790項分析
已完成77805項分析
已完成77820項分析
已完成77835項分析
已完成77850項分析
已完成77865項分析
已完成77880項分析
已完成77895項分析
已完成77910項分析
已完成77925項分析
已完成77940項分析
已完成77955項分析
已完成77970項分析
已完成77985項分析
已完成78000項分析
已完成78015項分析
已完成78030項分析
已完成78045項分析
已完成78060項分析
已完成78075項分析
已完成78090項分析
已完成78105項分析
已完成78120項分析
已完成78135項分析
已完成78150項分析
已完成78165項分析
已完成78180項分析
已完成78195項分析
已完成78210項分析
已完成78225項分析
已完成78240項分析
已完成78255項分析
已完成78270項分析
已完成78285項分析
已完成78300項分析
已完成78315項分析
已完成78330項分析
已完成78345項分析
已完成78360項分析
已完成78375項分析
已完成78390項分析
已完成78405項分析
已完成78420項分析
已完成78435項分析
已完成78450項分析
已完成78465項分析
已完成78480項分析
已完成78495項分析
已完成78510項分析
已完成78525項分析
已完成78540項分析
已完成78555項分析
已完成78570項分析
已完成78585項分析
已完成78600項分析
已完成78615項分析
已完成78630項分析
已完成78645項分析
已完成78660項分析
已完成78675項分析
已完成78690項分析
已完成78705項分析
已完成7

已完成87720項分析
已完成87735項分析
已完成87750項分析
已完成87765項分析
已完成87780項分析
已完成87795項分析
已完成87810項分析
已完成87825項分析
已完成87840項分析
已完成87855項分析
已完成87870項分析
已完成87885項分析
已完成87900項分析
已完成87915項分析
已完成87930項分析
已完成87945項分析
已完成87960項分析
已完成87975項分析
已完成87990項分析
已完成88005項分析
已完成88020項分析
已完成88035項分析
已完成88050項分析
已完成88065項分析
已完成88080項分析
已完成88095項分析
已完成88110項分析
已完成88125項分析
已完成88140項分析
已完成88155項分析
已完成88170項分析
已完成88185項分析
已完成88200項分析
已完成88215項分析
已完成88230項分析
已完成88245項分析
已完成88260項分析
已完成88275項分析
已完成88290項分析
已完成88305項分析
已完成88320項分析
已完成88335項分析
已完成88350項分析
已完成88365項分析
已完成88380項分析
已完成88395項分析
已完成88410項分析
已完成88425項分析
已完成88440項分析
已完成88455項分析
已完成88470項分析
已完成88485項分析
已完成88500項分析
已完成88515項分析
已完成88530項分析
已完成88545項分析
已完成88560項分析
已完成88575項分析
已完成88590項分析
已完成88605項分析
已完成88620項分析
已完成88635項分析
已完成88650項分析
已完成88665項分析
已完成88680項分析
已完成88695項分析
已完成88710項分析
已完成88725項分析
已完成88740項分析
已完成88755項分析
已完成88770項分析
已完成88785項分析
已完成88800項分析
已完成88815項分析
已完成88830項分析
已完成88845項分析
已完成88860項分析
已完成88875項分析
已完成88890項分析
已完成88905項分析
已完成88920項分析
已完成88935項分析
已完成88950項分析
已完成8

已完成97965項分析
已完成97980項分析
已完成97995項分析
已完成98010項分析
已完成98025項分析
已完成98040項分析
已完成98055項分析
已完成98070項分析
已完成98085項分析
已完成98100項分析
已完成98115項分析
已完成98130項分析
已完成98145項分析
已完成98160項分析
已完成98175項分析
已完成98190項分析
已完成98205項分析
已完成98220項分析
已完成98235項分析
已完成98250項分析
已完成98265項分析
已完成98280項分析
已完成98295項分析
已完成98310項分析
已完成98325項分析
已完成98340項分析
已完成98355項分析
已完成98370項分析
已完成98385項分析
已完成98400項分析
已完成98415項分析
已完成98430項分析
已完成98445項分析
已完成98460項分析
已完成98475項分析
已完成98490項分析
已完成98505項分析
已完成98520項分析
已完成98535項分析
已完成98550項分析
已完成98565項分析
已完成98580項分析
已完成98595項分析
已完成98610項分析
已完成98625項分析
已完成98640項分析
已完成98655項分析
已完成98670項分析
已完成98685項分析
已完成98700項分析
已完成98715項分析
已完成98730項分析
已完成98745項分析
已完成98760項分析
已完成98775項分析
已完成98790項分析
已完成98805項分析
已完成98820項分析
已完成98835項分析
已完成98850項分析
已完成98865項分析
已完成98880項分析
已完成98895項分析
已完成98910項分析
已完成98925項分析
已完成98940項分析
已完成98955項分析
已完成98970項分析
已完成98985項分析
已完成99000項分析
已完成99015項分析
已完成99030項分析
已完成99045項分析
已完成99060項分析
已完成99075項分析
已完成99090項分析
已完成99105項分析
已完成99120項分析
已完成99135項分析
已完成99150項分析
已完成99165項分析
已完成99180項分析
已完成99195項分析
已完成9

In [70]:
import json
with open("taiwan.json","w") as f:
    json.dump(wowDiensData,f)

In [71]:

print(len([i for i in wowDiensData if 'areaRadius_Analyze' in i]))
x=[i for i in wowDiensData if 'avgCost_Analyze' in i and i['Nhuman_Analyze']>0]
print(len(x))
# x

15462
1076


In [74]:
for i in wowDiensData:
    i["_id"]="{}_{}".format(i['lat'],i['lng'])
client = pymongo.mongo_client.MongoClient("localhost", 27017, username='j122085', password='850605')
collection = client.rawData.taiwanInfo
ids=[data.pop("_id") for data in wowDiensData]
operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,wowDiensData)]
try:
    collection.bulk_write(operations ,ordered=False)
except Exception as er:
    print(er)

alldata=list(collection.find({}))

client.close()